In [24]:
import itertools
import pandas as pd

### Get all required primes $p$, with $2 \leq p < 10^7$

In [4]:
def rwh_primes2(n):
    # https://stackoverflow.com/questions/2068372/fastest-way-to-list-all-primes-below-n-in-python/3035188#3035188
    """ Input n>=6, Returns a list of primes, 2 <= p < n """
    n, correction = n - n % 6 + 6, 2 - (n % 6 > 1)
    sieve = bytearray([True]) * (n // 3)
    for i in range(1, int(n ** 0.5) // 3 + 1):
        if sieve[i]:
            k = 3 * i + 1 | 1
            sieve[k * k // 3::2 * k] = bytearray([False]) * ((n // 6 - k * k // 6 - 1) // k + 1)
            sieve[k * (k - 2 * (i & 1) + 4) // 3::2 * k] = bytearray([False]) * (
                    (n // 6 - k * (k - 2 * (i & 1) + 4) // 6 - 1) // k + 1)
    return [2, 3] + [3 * i + 1 | 1 for i in range(1, n // 3 - correction) if sieve[i]]

list_of_primes = rwh_primes2(10 ** 7)
len(list_of_primes)

664579

### Maintain a table to track digit positions for each prime number as well as its length in digits

In [23]:
prime_digit_positions_df = pd.DataFrame(list_of_primes, columns=['p']).astype('string')
prime_digit_positions_df['length'] = prime_digit_positions_df.p.str.len()
for pos in range(1, 10):
    prime_digit_positions_df.loc[prime_digit_positions_df.length >= 10 - pos, 10 - pos] = prime_digit_positions_df[prime_digit_positions_df.length >= 10 - pos].p.apply(lambda p_s: p_s[len(p_s) - 10 + pos])
    
prime_digit_positions_df.fillna('', inplace=True)
prime_digit_positions_df

,p,length,9,8,7,6,5,4,3,2,1
0,2,1,,,,,,,,,2
1,3,1,,,,,,,,,3
2,5,1,,,,,,,,,5
3,7,1,,,,,,,,,7
4,11,2,,,,,,,,1,1
...,...,...,...,...,...,...,...,...,...,...,...
664574,9999937,7,,,9,9,9,9,9,3,7
664575,9999943,7,,,9,9,9,9,9,4,3
664576,9999971,7,,,9,9,9,9,9,7,1
664577,9999973,7,,,9,9,9,9,9,7,3


### Iterate through the table, broken down by length $l$ of the primes, and for each length, iterate through the combinations of masking at most $l-1$ digits. In each sub-iteration, ensure that the non-masked digits are the same in the exact same positions via a group-by operation and that the masked digit positions all contain the exact same digit for every prime in a group. 

In [55]:
is_found = False
for length in range(2, 10):
    length_primes_df = prime_digit_positions_df[prime_digit_positions_df.length == length]
    for k in range(1, length):
        for hide_comb in itertools.combinations(range(1, length + 1), k):
            same_digit_condition = length_primes_df.p.apply(lambda _: True)
            if k > 1:
                for i in range(k - 1):
                    same_digit_condition = same_digit_condition & (length_primes_df[hide_comb[i]] == length_primes_df[hide_comb[i + 1]])
            comb_groups = length_primes_df[same_digit_condition].groupby([col for col in range(1, length + 1) if col not in hide_comb]).apply(lambda x_df: (x_df.p.to_list(), len(x_df)))
            max_comb_group = max(comb_groups, key=lambda t: t[1]) 
            if max_comb_group[1] == 8:
                is_found = True
                print('Length:', length, 'k:', k)
                print('Max prime digit replacement combinations:', max_comb_group[0])
                break
        if is_found:
            break
    if is_found:
        break

Length: 6 k: 3
Max prime digit replacement combinations: ['121313', '222323', '323333', '424343', '525353', '626363', '828383', '929393']
